In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import accuracy_score, classification_report
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from sklearn.linear_model import LogisticRegression

In [2]:
df = pd.read_csv("train.csv")
df.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
X = df.drop(['label'], axis = 1)
y = df['label']

In [4]:
# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [5]:
# Normalize the pixel values
X_train = X_train/255.0
X_test = X_test/255.0

## Neural Networks

In [6]:
# Convert targets to categorical format (one-hot encoding)
y_train_cat = to_categorical(y_train.astype(int), 10)
y_test_cat = to_categorical(y_test.astype(int), 10)

In [7]:
# Build a simple feedforward neural network
nn_model = Sequential([
    Dense(512, activation='relu', input_shape=(784,)),
    Dropout(0.2),
    Dense(256, activation='relu'),
    Dropout(0.2),
    Dense(128, activation='relu'),
    Dropout(0.2),
    Dense(10, activation='softmax')  # 10 output classes
])

In [8]:
# Compile the model
nn_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [9]:
# Train the model with a validation split for monitoring
nn_model.fit(X_train, y_train_cat, validation_split=0.2, epochs=20, batch_size=32)

Epoch 1/20
840/840 [==============================] - 6s 6ms/step - loss: 0.3407 - accuracy: 0.8962 - val_loss: 0.1551 - val_accuracy: 0.9524
Epoch 2/20
840/840 [==============================] - 5s 6ms/step - loss: 0.1519 - accuracy: 0.9536 - val_loss: 0.1281 - val_accuracy: 0.9626
Epoch 3/20
840/840 [==============================] - 5s 6ms/step - loss: 0.1162 - accuracy: 0.9650 - val_loss: 0.1109 - val_accuracy: 0.9699
Epoch 4/20
840/840 [==============================] - 6s 7ms/step - loss: 0.0951 - accuracy: 0.9718 - val_loss: 0.1224 - val_accuracy: 0.9668
Epoch 5/20
840/840 [==============================] - 5s 6ms/step - loss: 0.0798 - accuracy: 0.9763 - val_loss: 0.1065 - val_accuracy: 0.9707
Epoch 6/20
840/840 [==============================] - 5s 6ms/step - loss: 0.0648 - accuracy: 0.9800 - val_loss: 0.1239 - val_accuracy: 0.9688
Epoch 7/20
840/840 [==============================] - 5s 6ms/step - loss: 0.0633 - accuracy: 0.9804 - val_loss: 0.1097 - val_accuracy: 0.9751
Epoch 

In [10]:
# Predict the classes for the test set
y_pred = nn_model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(y_test_cat, axis=1)

print("Accuracy:", accuracy_score(y_true_classes, y_pred_classes))
print("Classification Report:\n", classification_report(y_true_classes, y_pred_classes))

263/263 [==============================] - 1s 2ms/step
Accuracy: 0.9739285714285715
Classification Report:
               precision    recall  f1-score   support

           0       0.99      0.99      0.99       816
           1       0.99      0.99      0.99       909
           2       0.95      0.97      0.96       846
           3       0.96      0.97      0.96       937
           4       0.97      0.98      0.98       839
           5       0.96      0.97      0.96       702
           6       0.99      0.98      0.98       785
           7       0.97      0.97      0.97       893
           8       0.98      0.97      0.97       835
           9       0.99      0.95      0.97       838

    accuracy                           0.97      8400
   macro avg       0.97      0.97      0.97      8400
weighted avg       0.97      0.97      0.97      8400



## Support Vector Classifier

In [11]:
# Create an SVM model with RBF kernel
svm_model = SVC(kernel='rbf', C=1.0, gamma='scale')  # Adjust hyperparameters as needed

In [12]:
# Train the SVM model
svm_model.fit(X_train, y_train)

SVC()

In [13]:
# Test the SVM model
y_pred = svm_model.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.973452380952381
Classification Report:
               precision    recall  f1-score   support

           0       0.99      0.99      0.99       816
           1       0.98      0.99      0.99       909
           2       0.98      0.97      0.97       846
           3       0.97      0.96      0.96       937
           4       0.96      0.97      0.97       839
           5       0.97      0.97      0.97       702
           6       0.98      0.99      0.98       785
           7       0.98      0.97      0.97       893
           8       0.97      0.97      0.97       835
           9       0.96      0.96      0.96       838

    accuracy                           0.97      8400
   macro avg       0.97      0.97      0.97      8400
weighted avg       0.97      0.97      0.97      8400



## Random Forest

In [14]:
# Create a Random Forest model
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)  # 100 trees

In [15]:
# Train the Random Forest model
rf_model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [16]:
# Test the Random Forest model
y_pred = rf_model.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.9629761904761904
Classification Report:
               precision    recall  f1-score   support

           0       0.98      0.98      0.98       816
           1       0.98      0.99      0.99       909
           2       0.96      0.96      0.96       846
           3       0.96      0.95      0.96       937
           4       0.96      0.97      0.96       839
           5       0.96      0.96      0.96       702
           6       0.96      0.98      0.97       785
           7       0.97      0.95      0.96       893
           8       0.95      0.95      0.95       835
           9       0.93      0.94      0.94       838

    accuracy                           0.96      8400
   macro avg       0.96      0.96      0.96      8400
weighted avg       0.96      0.96      0.96      8400



## XGBoost

In [17]:
# Create an XGBoost Classifier
xgb_model = xgb.XGBClassifier(
    n_estimators=100,  # Number of trees
    learning_rate=0.1,  # Learning rate
    max_depth=6,  # Maximum depth of each tree
    subsample=1.0,  # Fraction of samples to use for each tree
    colsample_bytree=1.0,  # Fraction of features to use for each tree
    random_state=42
)

In [18]:
# Train the XGBoost model
xgb_model.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=1.0, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=6, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [19]:
# Test the model on the test set
y_pred = xgb_model.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.9647619047619047
Classification Report:
               precision    recall  f1-score   support

           0       0.99      0.98      0.99       816
           1       0.98      0.99      0.99       909
           2       0.97      0.96      0.96       846
           3       0.97      0.94      0.95       937
           4       0.96      0.97      0.96       839
           5       0.96      0.95      0.96       702
           6       0.97      0.98      0.98       785
           7       0.97      0.95      0.96       893
           8       0.95      0.97      0.96       835
           9       0.93      0.96      0.94       838

    accuracy                           0.96      8400
   macro avg       0.96      0.96      0.96      8400
weighted avg       0.96      0.96      0.96      8400



## Logistic Regression

In [20]:
# Logistic Regression for Nominal Targets
log_reg_model = LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=1000)


In [21]:
# Train the Model
log_reg_model.fit(X_train, y_train)

LogisticRegression(max_iter=1000, multi_class='multinomial')

In [22]:
# Predict and evaluate
y_pred = log_reg_model.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.9189285714285714
Classification Report:
               precision    recall  f1-score   support

           0       0.96      0.96      0.96       816
           1       0.96      0.98      0.97       909
           2       0.91      0.89      0.90       846
           3       0.91      0.87      0.89       937
           4       0.92      0.93      0.93       839
           5       0.85      0.88      0.87       702
           6       0.93      0.96      0.94       785
           7       0.93      0.92      0.93       893
           8       0.90      0.88      0.89       835
           9       0.91      0.90      0.91       838

    accuracy                           0.92      8400
   macro avg       0.92      0.92      0.92      8400
weighted avg       0.92      0.92      0.92      8400

